# 事前準備①：データセットの取得
データセットを本Colab上にダウンロードするには以下のどちらかを実行ください。

## [Option1] Google Driveから自身のPCに一度ダウンロードしてから、本Colabへアップロードする
Google DriveのURLはこちらです。

https://drive.google.com/file/d/16P1sryuVWHOTR9NdCLzRTinGUgokqPdg/view?usp=share_link

## [Option2] Google Driveから本Colabへ直接ダウンロードする
下記コマンドを実行する。

（参考）https://qiita.com/namakemono/items/c963e75e0af3f7eed732

In [ ]:
!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=16P1sryuVWHOTR9NdCLzRTinGUgokqPdg" > /dev/null
!CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=16P1sryuVWHOTR9NdCLzRTinGUgokqPdg" -o archive.zip
!unzip -q archive.zip



---



# 事前準備②：学習済みモデルの取得
データセットを本Colab上にダウンロードするには以下のどちらかを実行ください。

## [Option1] Google Driveから自身のPCに一度ダウンロードしてから、本Colabへアップロードする
Google DriveのURLはこちらです。

https://drive.google.com/file/d/14PtYuFZc-5sB2n9lLUDku8bgyEKSLZG5/view?usp=share_link

## [Option2] Google Driveから本Colabへ直接ダウンロードする
下記コマンドを実行する。

（参考）https://qiita.com/namakemono/items/c963e75e0af3f7eed732

In [ ]:
!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=14PtYuFZc-5sB2n9lLUDku8bgyEKSLZG5" > /dev/null
!CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"
!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=14PtYuFZc-5sB2n9lLUDku8bgyEKSLZG5" -o drone_Trained.pth



---



# ここから推論実行用のプログラム

## インポートライブラリ

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

import statistics

from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

!pip install -q segmentation-models-pytorch
!pip install -q torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## パス設定
画像、モデル、出力画像の保存先のパス指定

In [ ]:
IMAGE_PATH = #ドローン画像が格納されているフォルダのパス（相対パスでも絶対パスでもOK）
MODEL_PATH = #学習済みモデルファイルへのパス（相対パスでも絶対パスでもOK）

# モデルの推論結果のマスク画像を格納するフォルダを作成およびパスを変数にセット
!mkdir -p Model_Infer_Result
SAVE_PATH = 'Model_Infer_Result'

## データセットの構築
画像ファイル名のリストを作成します。

In [ ]:
def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename)
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df = create_df()
X_test = df['id'].values
print('Dataset Size    : ', len(X_test))

## mappingのkeyとvalueの反転(visualize()で使用)

In [ ]:
reverse_mapping = {1: (150, 143, 9),
                   2: (0, 0, 0),
                   3: (0, 0, 0),
                   4: (0, 0, 0),
                   5: (0, 0, 0),
                   6: (0, 0, 0),
                   7: (0, 0, 0),
                   8: (0, 0, 0),
                   9: (0, 0, 0),
                   10: (0, 0, 0),
                   11: (0, 0, 0),
                   12: (0, 0, 0),
                   13: (0, 0, 0),
                   14: (0, 0, 0),
                   15: (0, 0, 0),
                   16: (0, 0, 0),
                   17: (0, 0, 0),
                   18: (0, 0, 0),
                   19: (0, 0, 0),
                   20: (0, 0, 0),
                   21: (0, 0, 0),
                   22: (0, 0, 0),
                   23: (0, 0, 0),
                   0: (0, 0, 0),
            }      

## 2次元の出力を3次元のRGB画像に変換(value値からmappingに定義されたRGB値に変換)

In [ ]:
def visualize(temp):
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0,len(reverse_mapping)):
        r[temp==l]=reverse_mapping[l][0]
        g[temp==l]=reverse_mapping[l][1]
        b[temp==l]=reverse_mapping[l][2]

    rgb = np.zeros((temp.shape[1], temp.shape[2],3))

    rgb[:,:,0] = (r)
    rgb[:,:,1] = (g)
    rgb[:,:,2] = (b)
    return rgb

## テスト時のデータセットの定義


In [ ]:
class DroneTestDataset(Dataset):
    
    def __init__(self, img_path, X, transform=None):
        self.img_path = img_path
        self.X = X
        self.transform = transform
      
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(self.img_path, self.X[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        filename = str(self.X[idx])
        
        if self.transform is not None:
            aug = self.transform(image=img)
            img = Image.fromarray(aug['image'])
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        return img, filename

## データセットの変形(Data Augmentation)


In [ ]:
t_test = A.Resize(768, 1152, interpolation=cv2.INTER_NEAREST)
test_set = DroneTestDataset(IMAGE_PATH, X_test, transform=t_test)

## モデルのロード＆事前学習済みのパラメータのロード

In [ ]:
model = smp.Unet('mobilenet_v2', encoder_weights='imagenet', classes=24, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16])
model.load_state_dict(torch.load(MODEL_PATH))

## 推論実行（全画像一括実行）

In [ ]:
def predict_image(model, image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    model.eval()
    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(image)
    model.to(device); image=image.to(device)
    with torch.no_grad():
        image = image.unsqueeze(0)
        
        output = model(image)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked

In [ ]:
for i in tqdm(range(len(test_set))):
    img, filename= test_set[i]
    pred_mask = predict_image(model, img)
    pred_mask = pred_mask.cpu().numpy().copy()
    pred_mask = pred_mask.reshape(1,pred_mask.shape[0],pred_mask.shape[1])
    cv2.imwrite(os.path.join(SAVE_PATH, filename + ".png"), visualize(pred_mask), [cv2.IMWRITE_PNG_COMPRESSION,9])#ここで画像を保存

## 推論実行（画像をランダムに1枚実行）

In [ ]:
import random
index = random.randint(0, len(test_set))
img, filename = test_set[index]
pred_mask = predict_image(model, img)
pred_mask = pred_mask.cpu().numpy().copy()
pred_mask = pred_mask.reshape(1, pred_mask.shape[0], pred_mask.shape[1])
print(filename)


import matplotlib.pyplot as plt
plt.figure(figsize=(20,16))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(visualize(pred_mask))

## 推論実行（画像を任意に1枚実行）

In [ ]:
filepath = os.path.join(IMAGE_PATH, 'DJI_0075.jpg')
img = cv2.imread(filepath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

aug = t_test(image=img)
img = Image.fromarray(aug['image'])

pred_mask = predict_image(model, img)
pred_mask = pred_mask.cpu().numpy().copy()
pred_mask = pred_mask.reshape(1, pred_mask.shape[0], pred_mask.shape[1])

print(filepath)

%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,16))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(visualize(pred_mask))